In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

In [3]:
df = pd.read_csv('data.csv')[['Age','Fare','SibSp','Parch','Survived']]

In [4]:
df.sample(3)

,Age,Fare,SibSp,Parch,Survived
870,26.0,7.8958,0,0,0
855,18.0,9.3500,0,1,1
35,42.0,52.0000,1,0,0


In [5]:
df.dropna(inplace = True)

In [6]:
df['Family'] = df['SibSp']+df['Parch']

In [7]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [8]:
df.sample(3)

,Age,Fare,Survived,Family
777,5.0,12.4750,1,0
737,35.0,512.3292,1,0
418,30.0,13.0000,0,0


In [9]:
X = df.iloc[:,[0,1,3]]
y = df.iloc[:,2]


In [12]:
X

,Age,Fare,Family
0,22.0,7.2500,1
1,38.0,71.2833,1
2,26.0,7.9250,0
3,35.0,53.1000,1
4,35.0,8.0500,0
...,...,...,...
885,39.0,29.1250,5
886,27.0,13.0000,0
887,19.0,30.0000,0
889,26.0,30.0000,0


In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [22]:
X_train.sample(3)

,Age,Fare,Family
403,28.0,15.8500,1
22,15.0,8.0292,0
365,30.0,7.2500,0


In [23]:
# without binarization
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [16]:
accuracy_score(y_test,y_pred)

0.6293706293706294

In [17]:
print(np.mean(cross_val_score(clf,X,y,cv=10,scoring='accuracy')))

0.6443270735524257


In [18]:
#applying binarization

from sklearn.preprocessing import Binarizer


In [19]:
trf = ColumnTransformer([
    ('bin',Binarizer(copy=False),['Family'])
],remainder='passthrough')

### binarizer will make the family column either 0 or 1 based upon the person is alone or not

In [20]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [31]:
X_train_trf
X_train_trf.shape

(571, 3)

In [41]:
trf.named_transformers_

{'bin': Binarizer(copy=False), 'remainder': 'passthrough'}

In [42]:
pd.DataFrame(X_train_trf,columns=['Family','Age','Fare'])

,Family,Age,Fare
0,0.0,30.0,13.0000
1,1.0,36.0,17.4000
2,1.0,60.0,39.0000
3,1.0,8.0,21.0750
4,1.0,9.0,31.2750
...,...,...,...
566,1.0,17.0,7.0542
567,0.0,28.0,9.5000
568,1.0,18.0,7.8542
569,0.0,24.0,7.1417


In [43]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)

y_pred2 = clf.predict(X_test_trf)

accuracy_score(y_test,y_pred2)

0.6363636363636364

In [44]:
X_trf = trf.fit_transform(X)

In [47]:
np.mean(cross_val_score(clf,X_trf,y,scoring='accuracy',cv=10))

0.6290297339593114